In [125]:
import os
import numpy as np
mass=8000
channel="WW"
profile='NFW'
set="0000"

Ntrials = 10000
Njobs = 10
SignalSub = False

sys.path.append("/data/user/tchau/Sandbox/GC_OscNext/DMfit/DMfit")
sys.path.append("/data/user/tchau/Sandbox/GC_OscNext/PDFs")
sys.path.append("/data/user/tchau/Sandbox/GC_OscNext/DetResponse")
sys.path.append("/data/user/tchau/Sandbox/GC_OscNext/Utils")

from Detector import *
from Signal import *
from Plot_Histogram import *

from modeling import PdfBase, Model, Parameter
from data import DataSet
from llh import LikelihoodRatioTest
from scipy.interpolate import interp1d


In [126]:
def UpperLimit_Xi(Rate, Rate_Scr, Bkg, Data, LLH="SignalSub", CL=1.64, deltaxi=5e-5):
    SignalPDF = PdfBase(Rate.flatten()/np.sum(Rate.flatten()), name="SignalPDF")
    ScrSignalPDF = PdfBase(Rate_Scr.flatten()/np.sum(Rate_Scr.flatten()), name="ScrSignalPDF")
    BkgPDF = PdfBase(Bkg.flatten()/np.sum(Bkg.flatten()), name="Bkg")
    DataPDF = PdfBase(Data.flatten()/np.sum(Data.flatten()), name="Data")


    sig_fit = Parameter(value=0.001, limits=(0,1), fixed=False, name="fit")
    sig_fix = Parameter(value=0.0, limits=(0,1), fixed=True, name="fix")
    sig_inj = Parameter(value=0.0, limits=(0,1), fixed=True, name="inj")

    # signal subtraction
    if LLH=="SignalSub":
        modelH1 = (sig_fit* SignalPDF) + (1-sig_fit)*(BkgPDF) + sig_fit* BkgPDF - sig_fit*ScrSignalPDF
        modelH0 = (sig_fix* SignalPDF) + (1-sig_fix)*(BkgPDF) + sig_fix* BkgPDF - sig_fix*ScrSignalPDF
    elif LLH=="Poisson":
    # # in case of normal likelihood:
        modelH1 = (sig_fit* SignalPDF) + (1-sig_fit)*(BkgPDF)
        modelH0 = (sig_fix* SignalPDF) + (1-sig_fix)*(BkgPDF)

    lr = LikelihoodRatioTest(model = modelH1, null_model = modelH0)
    # pseudo_data = (sig_inj* SignalPDF) + (1-sig_inj)*(BkgPDF) + sig_inj* BkgPDF - sig_inj*ScrSignalPDF
    data = DataSet()
    data.asimov(np.sum(Data), DataPDF)
    lr.data = data

    T = np.array([0])
    x = deltaxi
    xi = np.array([0])
    while T[-1]<CL*1.5:
        xi = np.append(xi,x)
        lr.models['H0'].parameters["fix"].value = x
        lr.fit("H0")
        lr.fit("H1")
        if lr.models['H1'].parameters["fit"].value > x:
            T = np.append(T, 0)
        else:    
            T = np.append(T, lr.TS)
        x+=deltaxi
        
    
    f = interp1d(T, xi)
    xi_CL = f(CL)
    return xi_CL



In [118]:
Bin = Std_Binning(1400)
Reco = RecoRate(channel, 
            mass, 
            profile, 
            Bin,
            set=set)
Reco_Scr = RecoRate(channel, 
            mass, 
            profile, 
            Bin,
            Scramble=True,
            set=set)

Rate = Reco.ComputeRecoRate()
Rate_Scr = Reco_Scr.ComputeRecoRate()
Bkg_bwISJ = ScrambleBkg(Bin, bw="ISJ", oversample=10)


********************
Buiding final reco rate using Resp method
********************
Computing true rate with Charon spectra
channel: WW || mass: 8000 || profile: NFW || process: ann

Initial Flux Loading: /home/tchau/.local/lib/python3.10/site-packages/charon-0.1.0-py3.10.egg/charon/data/SpectraEW.hdf5
********************
Computing Response Matrix
********************
Accessing MC set 0000
********************
Buiding final reco rate using Resp method
********************
Computing true rate with Charon spectra
channel: WW || mass: 8000 || profile: NFW || process: ann

Initial Flux Loading: /home/tchau/.local/lib/python3.10/site-packages/charon-0.1.0-py3.10.egg/charon/data/SpectraEW.hdf5
********************
Computing Response Matrix
********************
Accessing MC set 0000
Loading file: 
/data/user/niovine/projects/DarkMatter_OscNext/Samples/OscNext/L7/Burnsample/OscNext_Level7_v02.00_burnsample_2012_pass2_variables_NoCut.pkl

Loading file: 
/data/user/niovine/projects/DarkMatter_O

In [119]:
np.sum(Rate)

9174889197930322.0

In [127]:
Burnsample = 10* DataHist(Bin)

signal = UpperLimit_Xi(Rate, Rate_Scr, Bkg_bwISJ, Burnsample, LLH="Poisson", CL=1.64, deltaxi=5e-5)
print(signal)
# signal = 0.0

Loading file: 
/data/user/niovine/projects/DarkMatter_OscNext/Samples/OscNext/L7/Burnsample/OscNext_Level7_v02.00_burnsample_2012_pass2_variables_NoCut.pkl

Loading file: 
/data/user/niovine/projects/DarkMatter_OscNext/Samples/OscNext/L7/Burnsample/OscNext_Level7_v02.00_burnsample_2013_pass2_variables_NoCut.pkl

Loading file: 
/data/user/niovine/projects/DarkMatter_OscNext/Samples/OscNext/L7/Burnsample/OscNext_Level7_v02.00_burnsample_2014_pass2_variables_NoCut.pkl

Loading file: 
/data/user/niovine/projects/DarkMatter_OscNext/Samples/OscNext/L7/Burnsample/OscNext_Level7_v02.00_burnsample_2015_pass2_variables_NoCut.pkl

Loading file: 
/data/user/niovine/projects/DarkMatter_OscNext/Samples/OscNext/L7/Burnsample/OscNext_Level7_v02.00_burnsample_2016_pass2_variables_NoCut.pkl

Loading file: 
/data/user/niovine/projects/DarkMatter_OscNext/Samples/OscNext/L7/Burnsample/OscNext_Level7_v02.00_burnsample_2017_pass2_variables_NoCut.pkl

Loading file: 
/data/user/niovine/projects/DarkMatter_OscN

/cvmfs/icecube.opensciencegrid.org/py3-v4.2.1/RHEL_7_x86_64/lib/python3.10/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [122]:

cfig_file = '/home/tchau/code/condor_submit/TSdist/dagman.config'
with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=500'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()
if SignalSub:
    sub_file = '/home/tchau/code/condor_submit/TSdist/TSdist_SignalSubLLH.submit'
else:
    sub_file = '/home/tchau/code/condor_submit/TSdist/TSdist_NormalLLH.submit'

with open(sub_file, 'w') as s:
    s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Sensitivity/TS_distribution.py'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write('logdir = /scratch/tchau/DM_OscNext/MakePDF/logs/'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 5GB'+'\n')
    s.write('\n')
    if SignalSub:
        s.write('arguments = -m $(mass) -c $(channel) -p $(profile) -s $(set) -i $(signal) -n $(Ntrials) -f $(file) -b $(bkgtrial) -l'+'\n')
    else:
        s.write('arguments = -m $(mass) -c $(channel) -p $(profile) -s $(set) -i $(signal) -n $(Ntrials) -f $(file) -b $(bkgtrial)'+'\n')

    s.write('queue')
    s.close()    

In [124]:
submit = '/home/tchau/code/condor_submit/TSdist/submit.sh'
path = '/home/tchau/code/condor_submit/TSdist/{}_{}'.format(channel, mass)
bkgtrial = 0
if not (os.path.exists(path)): os.makedirs(path)
with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    # for j in range(Njobs):
    for j in range(10):  
        dag_file = '{}/TSdist_job{}_bkgtrial{}_SignalSub{}_inject{}_{}_{}_{}.dag'.format(path, j, bkgtrial, SignalSub, signal, channel, mass, profile)
        with open(dag_file, 'w') as f:
            if SignalSub:
                f.write('JOB TSdist_job{}_bkgtrial{}_SignalSub{}_inject{}_{}_{}_{}.dag'.format(j, bkgtrial, SignalSub, signal, channel, mass, profile)+' TSdist_SignalSubLLH.submit \n')
            else:
                f.write('JOB TSdist_job{}_bkgtrial{}_SignalSub{}_inject{}_{}_{}_{}.dag'.format(j, bkgtrial, SignalSub, signal, channel, mass, profile)+' TSdist_NormalLLH.submit \n')
   
            f.write('VARS TSdist_job{}_bkgtrial{}_SignalSub{}_inject{}_{}_{}_{}.dag'.format(j, bkgtrial, SignalSub, signal, channel, mass, profile)+' JOBNAME="TSdist_job{}_bkgtrial{}_SignalSub{}_inject{}_{}_{}_{}.dag"'.format(j, bkgtrial, SignalSub, signal, channel, mass, profile) +' mass="{}" channel="{}" profile="{}" set="{}" signal="{}" Ntrials="{}" file="{}" bkgtrial="{}"'.format(mass, channel, profile, set, signal, Ntrials, j, bkgtrial)+'\n')
        f.close()
        s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
        print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_8000/TSdist_job0_bkgtrial0_SignalSubTrue_inject0.00042522100510430943_WW_8000_NFW.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_8000/TSdist_job1_bkgtrial0_SignalSubTrue_inject0.00042522100510430943_WW_8000_NFW.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_8000/TSdist_job2_bkgtrial0_SignalSubTrue_inject0.00042522100510430943_WW_8000_NFW.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_8000/TSdist_job3_bkgtrial0_SignalSubTrue_inject0.00042522100510430943_WW_8000_NFW.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_8000/TSdist_job4_bkgtrial0_SignalSubTrue_inject0.00042522100510430943_WW_8000_NFW.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_8000/TSdist_job5_bkgtrial0_SignalSubTrue_inject0.00042522100510430943_WW_8000_N